In [2]:
import os
import re
import datetime
import json
import glob
import jsonlines
import difflib

In [9]:
with open('prompts/pointwise_filter_prompt_collection_4_17.json', 'r', encoding='utf-8') as f:
     prompt_collection = json.load(f)

In [10]:
prompt_collection=prompt_collection[:7500]
with open('prompts/7500_pointwise_filter_prompt_collection_4_17.json', 'w', encoding='utf-8') as f:
    json.dump(prompt_collection, f, indent=4)

In [12]:
prompt_collection[0]

{'prompt_id': 1,
 'entity_unique_id': '0_United States_4_17',
 'prompt': '\n        <|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\n        You are an entity disambiguator. I\'ll provide you with a detailed description of entity disambiguation along with several tips on what to look out for:\n\n        1. Context: Look at the surrounding text to understand the topic.\n\n        2. Categories: Consider the type of the entity (person, organization, location, etc.).\n\n        3. Modifiers: Pay attention to words or phrases that add details to the mention.\n\n        4. Co-references: Check other mentions of the same entity in the text.\n\n        5. Temporal and Geographical Factors: Consider when and where the text was written.\n\n        6. External Knowledge: Use knowledge from outside the text.\n\n        Remember, effective entity disambiguation requires understanding the text thoroughly, having world knowledge, and exercising good judgment.\n \n        <|eot_id|><|st

In [3]:
OUTPUT_DIR   = "outputs/pointwise"
NUM_BATCHES  = 3

batch_files = sorted(glob.glob(f"{OUTPUT_DIR}/batch_*.json"))[:NUM_BATCHES]

all_outputs = []
for fpath in batch_files:
    with open(fpath, "r", encoding="utf-8") as fp:
        batch_rows = json.load(fp)          # list of result_dicts
        all_outputs.extend(batch_rows)

print(f"Loaded {len(all_outputs)} rows from {len(batch_files)} batch files.")

Loaded 22500 rows from 3 batch files.


In [4]:
all_outputs[0]

{'iteration': 1,
 'batch_number': 1,
 'prompt_id': 1,
 'entity_unique_id': '0_United States_4_17',
 'entity': 'United States',
 'candidate': {'mention': 'United States',
  'cand_name': 'United States',
  'cand_summary': "The United States of America is a federal republic and representative democracy, comprising 50 states, five major self-governing territories, and various possessions, with an estimated population of over 327 million people, the third most populous country in the world, and a highly developed country with the world's largest economy by nominal GDP, second-largest by purchasing power parity, and a leading political, cultural, and scientific force internationally.",
  'cand_text': "The United States of America (USA), commonly known as the United States (U.S. or US) or simply America, is a country comprising 50 states, a federal district, five major self-governing territories, and various possessions. At 3.8&nbsp;million square miles (9.8&nbsp;million km<sup>2</sup>), it i

### Mapping LLM outputs → filtered candidates per entity

In [5]:
negative_keywords = ['"no"', "'no'", "not related", "cannot establish a relationship"]

def extract_final_decision(text):
    for line in reversed(text.strip().splitlines()):
        if line.strip().startswith("Final Decision:"):
            return line.split("Final Decision:")[1].strip().lower()
    return None

In [8]:
output=all_outputs[0]
filter=False

decision = extract_final_decision(output["llm_pointwise_response"]) or "yes"
if any(neg in decision for neg in negative_keywords):
    filter=True

filter

False

In [19]:
results_by_entity = {}

for i, row in enumerate(all_outputs):
    print('len of row: ', len(row))
    print('row: ', row)
    decision = extract_final_decision(row["llm_pointwise_response"]) or "yes"
    
    # remove candidates for final decision - no 
    if any(neg in decision for neg in negative_keywords):
        continue
        
    uid = row["entity_unique_id"]
    print('uid: ', uid)
    if uid not in results_by_entity:
        results_by_entity[uid] = {
            "entity_title": row["entity"],
            "filtered_candidates": []
        }
    else:
        print('results_by_entity for ' + uid + ': ', results_by_entity[uid])
        
    if i==3: break
    results_by_entity[uid]["filtered_candidates"].append(row["candidate"])

len of row:  9
row:  {'iteration': 1, 'batch_number': 1, 'prompt_id': 1, 'entity_unique_id': '0_United States_4_17', 'entity': 'United States', 'candidate': {'mention': 'United States', 'cand_name': 'United States', 'cand_summary': "The United States of America is a federal republic and representative democracy, comprising 50 states, five major self-governing territories, and various possessions, with an estimated population of over 327 million people, the third most populous country in the world, and a highly developed country with the world's largest economy by nominal GDP, second-largest by purchasing power parity, and a leading political, cultural, and scientific force internationally.", 'cand_text': "The United States of America (USA), commonly known as the United States (U.S. or US) or simply America, is a country comprising 50 states, a federal district, five major self-governing territories, and various possessions. At 3.8&nbsp;million square miles (9.8&nbsp;million km<sup>2</s

In [20]:
results_by_entity['0_United States_4_17']

{'entity_title': 'United States',
 'filtered_candidates': [{'mention': 'United States',
   'cand_name': 'United States',
   'cand_summary': "The United States of America is a federal republic and representative democracy, comprising 50 states, five major self-governing territories, and various possessions, with an estimated population of over 327 million people, the third most populous country in the world, and a highly developed country with the world's largest economy by nominal GDP, second-largest by purchasing power parity, and a leading political, cultural, and scientific force internationally.",
   'cand_text': "The United States of America (USA), commonly known as the United States (U.S. or US) or simply America, is a country comprising 50 states, a federal district, five major self-governing territories, and various possessions. At 3.8&nbsp;million square miles (9.8&nbsp;million km<sup>2</sup>), it is the world's third or fourth largest country by total area and is slightly sma

### Filtered candidates list back to the original sed_outputs

In [13]:
knowledge_base = []
with jsonlines.open('/work/pi_wenlongzhao_umass_edu/8/KB with Summaries/main_dataset.jsonl') as reader:
    for obj in reader:
        knowledge_base.append(obj)

In [14]:
# Build an alias dictionary from the knowledge base that maps lowercased candidate mentions
alias_kb = {}
for entry in knowledge_base:
    key = entry['mention'].lower()
    alias_kb.setdefault(key, []).append(entry)

In [15]:
wn_salience_train = "/project/pi_wenlongzhao_umass_edu/8/data/WN-Salience-2/SED_train.json"
with open(wn_salience_train) as f:
    sed_outputs = json.load(f)

In [22]:
candidate_freq = {}
for alias, candidates in alias_kb.items():
    for cand in candidates:
        cand_id = cand.get('cand_wiki_id')
        candidate_freq[cand_id] = candidate_freq.get(cand_id, 0) + 1
        
max_freq = max(candidate_freq.values())
min_freq = min(candidate_freq.values())

In [17]:
def retrieve_candidates(mention_text, alias_dict, cutoff=0.8, alpha=0.5, num_candidates=10):
    mention_lower = mention_text.lower()
    all_keys = list(alias_dict.keys())
    close_keys = difflib.get_close_matches(mention_lower, all_keys, n=len(all_keys), cutoff=cutoff)
    # print(len(close_keys))
    # print(close_keys)
    
    candidates = []
    for key in close_keys:
        candidates.extend(alias_dict[key])
    print('length of candidates before removing duplicates: ', len(candidates))
    
    # Remove duplicates
    seen = set()
    unique_candidates = []
    for cand in candidates:
        # print('cand: ', cand)
        cand_id = cand.get('cand_wiki_id')
        if cand_id not in seen:
            unique_candidates.append(cand)
            seen.add(cand_id)
            
    print('length of candidates after removing duplicates: ', len(unique_candidates))

    if len(unique_candidates)>num_candidates:
        for candidate in unique_candidates:
            candidate_str = candidate.get('mention', '').lower()
            fuzzy_score = difflib.SequenceMatcher(None, mention_lower, candidate_str).ratio()
            # print('fuzzy_score: ', fuzzy_score)
            freq = candidate_freq.get(candidate.get('cand_wiki_id'), 0)
            if max_freq > min_freq:
                norm_freq = (freq - min_freq) / (max_freq - min_freq)
            else:
                norm_freq = 1
            # print('freq: ', freq)
            # candidate['weighted_score'] = fuzzy_score * (freq+1)
            candidate['weighted_score'] = (alpha*fuzzy_score) + ((1 - alpha)*norm_freq)
            # print('weighted score: ', candidate['weighted_score'])
        
        unique_candidates.sort(key=lambda cand: cand.get('weighted_score', 0), reverse=True)
    
    unique_candidates = unique_candidates[:num_candidates]
    
    return unique_candidates

In [ ]:
def get_entity_unique_id(article_index, entity):
    return f"{article_index}_{entity['entity_title']}_{entity['start']}_{entity['end']}"

for article_index, sed_output in enumerate(sed_outputs):
    for entity in sed_output["entities"]:
        entity_unique_id = get_entity_unique_id(article_index, entity)
        entity["entity_unique_id"] = entity_unique_id

        candidates = retrieve_candidates(entity["entity_title"], alias_kb)
        entity['candidates'] = candidates

length of candidates before removing duplicates:  1721
length of candidates after removing duplicates:  98
length of candidates before removing duplicates:  0
length of candidates after removing duplicates:  0
length of candidates before removing duplicates:  520
length of candidates after removing duplicates:  90
length of candidates before removing duplicates:  1838
length of candidates after removing duplicates:  182
length of candidates before removing duplicates:  520
length of candidates after removing duplicates:  90
length of candidates before removing duplicates:  1838
length of candidates after removing duplicates:  182
length of candidates before removing duplicates:  1721
length of candidates after removing duplicates:  98
length of candidates before removing duplicates:  0
length of candidates after removing duplicates:  0
length of candidates before removing duplicates:  878
length of candidates after removing duplicates:  109
length of candidates before removing duplicat

In [ ]:
sed_outputs[0]

In [ ]:
for article_idx, article in enumerate(sed_outputs):
    for entity in article["entities"]:
        uid = entity.get("entity_unique_id")
        entity["filtered_candidates"] = results_by_entity.get(uid, {}).get("filtered_candidates", [])

# print(json.dumps(sed_outputs, indent=2))
with open("outputs/pointwise/filtered_first3_batches.json", "w", encoding="utf-8") as fout:
    json.dump(sed_outputs, fout, indent=2)